In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 데이터 로드

In [4]:
import pandas as pd

In [6]:
path = '/content/drive/MyDrive/jbnu-swuni-ai'

train_o = pd.read_csv(path+"/train_data.csv")
test_o = pd.read_csv(path+"/test_data.csv")

train_p = pd.read_csv(path+"/merged_desc.csv")
test_p = pd.read_csv(path+"/merged_test.csv")

In [7]:
train = pd.merge(train_o, train_p, left_on='Filename', right_on='image', how='inner')
test= pd.merge(test_o, test_p,left_on='Filename', right_on='File', how='inner')

In [8]:
# 라벨 넘버링
label_mapping = {label: i for i, label in enumerate(train["label"].unique())}
num_labels = len(label_mapping)
train["label_num"] = train["label"].map(label_mapping)

## 부호, 불용어, 중복 제거

In [9]:
import re

# 부호 제거
def alpha_num(Title):
    return re.sub(r'[^A-Za-z0-9 ]', '', Title)

In [10]:
train['Title']=train['Title'].apply(alpha_num)
test['Title']=test['Title'].apply(alpha_num)

train['prompt']=train['prompt'].apply(alpha_num)
test['Prompt']=test['Prompt'].apply(alpha_num)

In [11]:
# 불용어 제거
def remove_stopwords(Title):
    final_text = []
    for i in Title.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

In [12]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [13]:
# 중복제거
def delete_duplication(text):
  words = text.split()

  unique_words = []

  for word in words:
    if word not in unique_words:
      unique_words.append(word)

  result = ' '.join(unique_words)
  return result

In [14]:
# train['Title'] = train['Title'].str.lower()
# test['Title'] = test['Title'].str.lower()
# train['Title'] = train['Title'].apply(alpha_num).apply(remove_stopwords)
# test['Title'] = test['Title'].apply(alpha_num).apply(remove_stopwords)

train['prompt'] = train['prompt'].str.lower()
test['Prompt'] = test['Prompt'].str.lower()
train['prompt'] = train['prompt'].apply(alpha_num).apply(remove_stopwords).apply(delete_duplication)
test['Prompt'] = test['Prompt'].apply(alpha_num).apply(remove_stopwords).apply(delete_duplication)

In [15]:
train = train[['id', 'Filename', 'Title', 'prompt', 'label', 'label_num']]
train.columns = ['id', 'Filename', 'Title', 'prompt', 'label', 'label_num']

In [16]:
test = test[['id', 'Filename', 'Title', 'Prompt']]
test.columns = ['id', 'Filename', 'Title', 'prompt']

In [17]:
train.to_csv(path+"/pre_train.csv")
test.to_csv(path+"/pre_test.csv")